In [1]:
import os
import shutil
import utils
import numpy as np
import random

from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision import datasets, models
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
utils.set_random_seed()

In [3]:
### Setting up "constants"
NUM_LABELS = 196
BATCH_SIZE = 64
LR = 2e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 25

In [4]:
simple_transforms = T.Compose([
    T.ToTensor(),
    T.Resize((224,244)),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(p=0.25),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    # T.GaussianBlur(kernel_size=(5,5)),
    # T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

resize_test = T.Compose([
    T.ToTensor(),
    T.Resize((224,224)),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [5]:
# set number of labels manually
NUM_LABELS = 196
BATCH_SIZE = 64
LR = 2e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 15
print(DEVICE)

cuda


In [6]:
%%capture
train_set = datasets.StanfordCars(
    "./stanfordcars/", 
    split="train", 
    transform=simple_transforms, 
    download=True
)
test_set = datasets.StanfordCars(
    "./stanfordcars/", 
    split="test", 
    transform=resize_test,
    download=True
)


train_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test_set, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    pin_memory=True
)

In [7]:
# Tensorboard config and writter init
PROJECT = "Assignment3"
TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", PROJECT)
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)

In [8]:
# cutmix implementation, implemented from pseudo code given by the paper

def CutMix(input, target):
    # get batch size to shuffle index of minibatch, and set a shuffled list
    B = list(range(input.shape[0]))
    shuffled_batch_idx = torch.tensor(random.sample(B, len(B)))

    # alg according to paper
    W = input[0].shape[1]
    H = input[0].shape[2]
    Lambda = torch.rand(1)
    r_x = torch.rand(1) * H
    r_y = torch.rand(1) * W
    r_w = torch.sqrt(1 - Lambda) * H
    r_h = torch.sqrt(1 - Lambda) * W

    x1 = int(torch.clamp((r_x - r_w / 2), min=0, max=W))
    x2 = int(torch.clamp((r_x + r_w / 2), min=0, max=W))
    y1 = int(torch.clamp((r_y - r_h / 2), min=0, max=H))
    y2 = int(torch.clamp((r_y + r_h / 2), min=0, max=H))
    
    # target = Lambda * target + (1 - Lambda) * target[shuffled_batch_idx]
    Lambda = 1 - ((x2-x1) * (y2 - y1) / (W*H))
    input[:, :, y1:y2, x1:x2] = input[shuffled_batch_idx, :, y1:y2, x1:x2]

    return input, target, shuffled_batch_idx, Lambda

In [20]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device=DEVICE):
    """ Training a model for one epoch """
    
    loss_list = []
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        prob_cutmix = torch.rand(1)
        if prob_cutmix > 0.75:
            # used for cutmix agumentation
            images, labels, shuffled_idx, Lambda = CutMix(images, labels)
            
            # Forward pass to get output/logits
            outputs = model(images)
            
            # Calculate Loss: softmax --> cross entropy loss
            # split loss values according to cutmix paper
            loss = criterion(outputs, labels) * Lambda  + criterion(outputs, labels[shuffled_idx]) * (1 - Lambda)
        else: 

            #compute output as usual
            outputs = model(images)
            loss = criterion(outputs, labels)

        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()

        progress_bar.set_description(f"Epoch {epoch+1} Iter {i+1}: loss {loss.item():.5f}. ")
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list


@torch.no_grad()
def eval_model(model, eval_loader, criterion, device=DEVICE):
    """ Evaluating the model for either validation or test """
    correct = 0
    total = 0
    loss_list = []
    
    for images, labels in eval_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass only to get logits/output
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss


def train_model(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs, device, tboard, start_epoch=0):
    """ Training a model for a given number of epochs"""
    
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
           
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = eval_model(
                    model=model, eval_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        tboard.add_scalar(f'Accuracy/Valid', accuracy, global_step=epoch+start_epoch)
        tboard.add_scalar(f'Loss/Valid', loss, global_step=epoch+start_epoch)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train_epoch(
                model=model, train_loader=train_loader, optimizer=optimizer,
                criterion=criterion, epoch=epoch, device=device
            )
        scheduler.step()
        train_loss.append(mean_loss)
        tboard.add_scalar(f'Loss/Train', mean_loss, global_step=epoch+start_epoch)

        loss_iters = loss_iters + cur_loss_iters
        
        # if(epoch % 5 == 0 or epoch==num_epochs-1):
        # print(f"Epoch {epoch+1}/{num_epochs}")
        # print(f"    Train loss: {round(mean_loss, 5)}")
        # print(f"    Valid loss: {round(loss, 5)}")
        # print(f"    Accuracy: {accuracy}%")
        # print("\n")
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc

In [21]:
%%capture
# Capture, cause pretrained param has a deprecated warning, but still works
# Loading models
vgg_model = models.vgg16_bn(pretrained=True)

In [22]:
vgg_model.classifier = nn.Linear(7*7*512, NUM_LABELS)

vgg_model.to(DEVICE);

In [23]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_vgg = torch.optim.Adam(vgg_model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler_vgg = torch.optim.lr_scheduler.StepLR(optimizer_vgg, step_size=7, gamma=0.1)

In [24]:
TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", "VGG16_tuned")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer_vgg = SummaryWriter(TBOARD_LOGS)

In [25]:
#model, optimizer, scheduler, criterion, train_loader, 
#    valid_loader, num_epochs, device, all_labels, tboard=None, start_epoch=0
#    ):
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=vgg_model, optimizer=optimizer_vgg, scheduler=scheduler_vgg, criterion=criterion,
        train_loader=train_loader, valid_loader=test_loader, num_epochs=20,
        device=DEVICE, tboard=writer_vgg
)

Epoch 2 Iter 81: loss 4.75970. :  63%|██████▎   | 81/128 [01:03<00:42,  1.11it/s]

In [15]:
#model, optimizer, scheduler, criterion, train_loader, 
#    valid_loader, num_epochs, device, all_labels, tboard=None, start_epoch=0
#    ):
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=vgg_model, optimizer=optimizer_vgg, scheduler=scheduler_vgg, criterion=criterion,
        train_loader=train_loader, valid_loader=test_loader, num_epochs=20,
        device=DEVICE, tboard=writer_vgg
)

Epoch 1 Iter 128: loss 4.50115. : 100%|██████████| 128/128 [01:44<00:00,  1.23it/s]


Epoch 1/20
    Train loss: 5.11658
    Valid loss: 5.29631
    Accuracy: 0.7088670563362767%




Epoch 2 Iter 128: loss 4.09720. : 100%|██████████| 128/128 [01:43<00:00,  1.23it/s]


Epoch 2/20
    Train loss: 4.25511
    Valid loss: 4.8006
    Accuracy: 4.340256187041413%




Epoch 3 Iter 128: loss 2.47387. : 100%|██████████| 128/128 [01:44<00:00,  1.23it/s]


...

Epoch 18/20
    Train loss: 1.28107
    Valid loss: 0.82264
    Accuracy: 77.00534759358288%




Epoch 19 Iter 128: loss 0.38047. : 100%|██████████| 128/128 [01:44<00:00,  1.23it/s]


Epoch 19/20
    Train loss: 1.3288
    Valid loss: 0.8172
    Accuracy: 77.41574431040915%




Epoch 20 Iter 128: loss 1.95745. : 100%|██████████| 128/128 [01:43<00:00,  1.23it/s]

Epoch 20/20
    Train loss: 1.30857
    Valid loss: 0.79827
    Accuracy: 77.92563114040543%


Training completed


In [ ]:
#model, optimizer, scheduler, criterion, train_loader, 
#    valid_loader, num_epochs, device, all_labels, tboard=None, start_epoch=0
#    ):
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=vgg_model, optimizer=optimizer_vgg, scheduler=scheduler_vgg, criterion=criterion,
        train_loader=train_loader, valid_loader=test_loader, num_epochs=20,
        device=DEVICE, tboard=writer_vgg
)

Epoch 1 Iter 128: loss 3.93485. : 100%|██████████| 128/128 [01:39<00:00,  1.29it/s]


Epoch 1/20
    Train loss: 4.86316
    Valid loss: 5.27226
    Accuracy: 0.9949011317000372%




Epoch 2 Iter 128: loss 3.42686. : 100%|██████████| 128/128 [01:40<00:00,  1.28it/s]


Epoch 2/20
    Train loss: 3.43878
    Valid loss: 4.00098
    Accuracy: 11.279691580649173%




Epoch 3 Iter 128: loss 2.44985. : 100%|██████████| 128/128 [01:40<00:00,  1.28it/s]


...

Epoch 18/20
    Train loss: 0.62912
    Valid loss: 0.79925
    Accuracy: 78.09973883845292%




Epoch 19 Iter 128: loss 0.41274. : 100%|██████████| 128/128 [01:40<00:00,  1.28it/s]


Epoch 19/20
    Train loss: 0.64678
    Valid loss: 0.7993
    Accuracy: 77.97537619699042%




Epoch 20 Iter 128: loss 0.33265. : 100%|██████████| 128/128 [01:40<00:00,  1.28it/s]

Epoch 20/20
    Train loss: 0.64386
    Valid loss: 0.79677
    Accuracy: 78.07486631016043%


Training completed
